# Early Stopping Callback
In this assignment you should write a callback that will check the training loss value at the end of each epoch and if it was less than a threshold it will stop training.

In [9]:
import tensorflow as tf

class EarlyStoppingByLossVal(tf.keras.callbacks.Callback):
    def __init__(self, monitor='loss', value=0.00001, verbose=0):
        super(EarlyStoppingByLossVal, self).__init__()
        self.monitor = monitor
        self.value   = value
        self.verbose = verbose
    def on_epoch_end(self, epoch, logs=None):
        if self.verbose == 1:
            print(" Epoch: {}, Train loss: {:.2f}".format(epoch, logs[self.monitor]))
        if logs[self.monitor] < self.value:
            print("Stopping training...")
            self.model.stop_training = True

## Model Training

In [3]:
import tensorflow_datasets as tfds
ds, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_ds, validation_ds, test_ds) = ds

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteB7RDZY/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteB7RDZY/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [4]:
IMAGE_SIZE = (150, 150)
def preprocess_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

BATCH_SIZE = 32
train_batches = train_ds.shuffle(num_examples // 4).map(preprocess_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_ds.map(preprocess_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_ds.map(preprocess_image).batch(1)

In [5]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_batches,
          epochs=10,
          validation_data=validation_batches,
          verbose=0,
          callbacks=[EarlyStoppingByLossVal(value=.5, verbose=1)])

 Epoch: 0, Train loss: 0.67
 Epoch: 1, Train loss: 0.63
 Epoch: 2, Train loss: 0.58
 Epoch: 3, Train loss: 0.52
 Epoch: 4, Train loss: 0.48
Stopping training...
